<a href="https://colab.research.google.com/github/123yaroslav/DL_MSU/blob/main/DL_MSU_HW07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Домашнее задание №7
### Генерация поэзии с помощью нейронных сетей: шаг 1
##### Авторы: [Радослав Нейчев](https://www.linkedin.com/in/radoslav-neychev/), Горохов Олег

Ваша основная задача: научиться генерировать стихи с помощью простой рекуррентной нейронной сети (Vanilla RNN). В качестве корпуса текстов для обучения будет выступать роман в стихах "Евгений Онегин" Александра Сергеевича Пушкина.

In [1]:
# Не меняйте блок кода ниже! Здесь указаны все необходимые import-ы
# __________start of block__________
import string
import os
from random import sample

import numpy as np
import torch, torch.nn as nn
import torch.nn.functional as F

from IPython.display import clear_output

import matplotlib.pyplot as plt
%matplotlib inline
# __________end of block__________

In [2]:
# Не меняйте блок кода ниже!
# __________start of block__________
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print('{} device is available'.format(device))
# __________end of block__________

cuda device is available


#### 1. Загрузка данных.

In [3]:
# do not change the code in the block below
# __________start of block__________
!wget https://raw.githubusercontent.com/MSUcourses/Data-Analysis-with-Python/main/Deep%20Learning/onegin_hw07.txt -O ./onegin.txt

with open('onegin.txt', 'r') as iofile:
    text = iofile.readlines()

text = "".join([x.replace('\t\t', '').lower() for x in text]) # Убираем лишние символы табуляций, приводим все буквы к нижнему регистру
# __________end of block__________

--2024-04-07 07:36:05--  https://raw.githubusercontent.com/MSUcourses/Data-Analysis-with-Python/main/Deep%20Learning/onegin_hw07.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 262521 (256K) [text/plain]
Saving to: ‘./onegin.txt’

./onegin.txt        100%[===================>] 256.37K  --.-KB/s    in 0.01s   

2024-04-07 07:36:05 (22.1 MB/s) - ‘./onegin.txt’ saved [262521/262521]



Выведем несколько первых символов входного текста. Видим, что символы табуляций удалены, буквы приведены к нижнему регистру. Символы \n мы оставляем - чтобы научить сеть генерировать символ \n, когда нужно перейти на новую строку.

In [4]:
text[:36]

'\ni\n\n«мой дядя самых честных правил,\n'

#### 2. Построение словаря и предобработка текста
В данном задании требуется построить языковую модель на уровне символов. Приведем весь текст к нижнему регистру и построим словарь из всех символов в доступном корпусе текстов. Также добавим технический токен `<sos>`.

In [5]:
# Не меняйте блок кода ниже!
# __________start of block__________
tokens = sorted(set(text.lower())) + ['<sos>'] # Строим множество всех токенов-символов и добавляем к нему служебный токен <sos>
num_tokens = len(tokens)

assert num_tokens == 84, "Check the tokenization process"

token_to_idx = {x: idx for idx, x in enumerate(tokens)} # Строим словарь с ключами-токенами и значениями-индексами в списке токенов
idx_to_token = {idx: x for idx, x in enumerate(tokens)} # Строим обратный словарь (чтобы по индексу можно было получить токен)

assert len(tokens) == len(token_to_idx), "Mapping should be unique"

print("Seems fine!")


text_encoded = [token_to_idx[x] for x in text]
# __________end of block__________

Seems fine!


__Ваша задача__: обучить классическую рекуррентную нейронную сеть (Vanilla RNN) предсказывать следующий символ на полученном корпусе текстов и сгенерировать последовательность длины 100 для фиксированной начальной фразы.

Вы можете воспользоваться кодом с занятия №6 или же обратиться к следующим ссылкам:
* Замечательная статья за авторством Andrej Karpathy об использовании RNN: [link](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
* Пример char-rnn от Andrej Karpathy: [github repo](https://github.com/karpathy/char-rnn)
* Замечательный пример генерации поэзии Шекспира: [github repo](https://github.com/spro/practical-pytorch/blob/master/char-rnn-generation/char-rnn-generation.ipynb)

Данное задание является достаточно творческим. Не страшно, если поначалу оно вызывает затруднения. Последняя ссылка в списке выше может быть особенно полезна в данном случае.

Далее для вашего удобства реализована функция, которая генерирует случайный батч размера `batch_size` из строк длиной `seq_length`. Вы можете использовать его при обучении модели.

In [6]:
# Не меняйте код ниже
# __________start of block__________
batch_size = 256 # Размер батча. Батч - это набор последовательностей символов.
seq_length = 100 # Максимальная длина одной последовательности символов в батче
start_column = np.zeros((batch_size, 1), dtype=int) + token_to_idx['<sos>'] # Добавляем в начало каждой строки технический символ - для определения начального состояния сети

def generate_chunk():
    global text_encoded, start_column, batch_size, seq_length

    start_index = np.random.randint(0, len(text_encoded) - batch_size*seq_length - 1) # Случайным образом выбираем индекс начального символа в батче
    # Строим непрерывный батч.
    # Для этого выбираем в исходном тексте подпоследовательность, которая начинается с индекса start_index и имеет размер batch_size*seq_length.
    # Затем мы делим эту подпоследовательность на batch_size последовательностей размера seq_length. Это и будет батч, матрица размера batch_size*seq_length.
    # В каждой строке матрицы будут указаны индексы
    data = np.array(text_encoded[start_index:start_index + batch_size*seq_length]).reshape((batch_size, -1))
    yield np.hstack((start_column, data))
# __________end of block__________

Пример батча:

In [7]:
next(generate_chunk())

array([[83,  1, 46, ...,  0, 52, 49],
       [83, 50, 62, ..., 51, 45, 56],
       [83, 64, 54, ..., 53,  1, 55],
       ...,
       [83, 58, 53, ..., 59, 57,  1],
       [83, 47, 62, ..., 50, 63, 73],
       [83,  1, 59, ...,  0, 47, 62]])

Далее вам предстоит написать код для обучения модели и генерации текста.

In [8]:
class CharRNN(nn.Module):
    def __init__(self, num_classes, embedding_dim=128, hidden_size=256, n_layers=1):
        super(CharRNN, self).__init__()
        self.num_classes = num_classes
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        self.n_layers = n_layers

        self.embedding = nn.Embedding(num_classes, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_size, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x, hidden):
        embedded = self.embedding(x)

        out, hidden = self.rnn(embedded, hidden)

        out = out.reshape(out.size(0)*out.size(1), out.size(2))

        out = self.fc(out)
        return out, hidden

    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_size).to(device)
        return hidden


В качестве иллюстрации ниже доступен график значений функции потерь, построенный в ходе обучения авторской сети (сам код для ее обучения вам и предстоит написать).

In [30]:

model = CharRNN(num_tokens, embedding_dim=128, hidden_size=256, n_layers=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

n_epochs = 10000

for epoch in range(n_epochs):
    hidden = model.init_hidden(batch_size)

    for batch in generate_chunk():
        inputs, targets = batch[:, :-1], batch[:, 1:]
        inputs, targets = torch.LongTensor(inputs).to(device), torch.LongTensor(targets).to(device)

        optimizer.zero_grad()

        output, hidden = model(inputs, hidden.detach())

        loss = criterion(output, targets.reshape(-1))
        loss.backward()

        optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch: {epoch}, Loss: {loss.item()}')



Epoch: 0, Loss: 4.442775726318359
Epoch: 10, Loss: 2.738335609436035
Epoch: 20, Loss: 2.4892454147338867
Epoch: 30, Loss: 2.3867037296295166
Epoch: 40, Loss: 2.3131661415100098
Epoch: 50, Loss: 2.2136638164520264
Epoch: 60, Loss: 2.183074951171875
Epoch: 70, Loss: 2.0953595638275146
Epoch: 80, Loss: 2.014157772064209
Epoch: 90, Loss: 2.0238277912139893
Epoch: 100, Loss: 1.979306936264038
Epoch: 110, Loss: 1.902870774269104
Epoch: 120, Loss: 1.9894479513168335
Epoch: 130, Loss: 1.9420669078826904
Epoch: 140, Loss: 1.7641280889511108
Epoch: 150, Loss: 1.7651162147521973
Epoch: 160, Loss: 1.746534824371338
Epoch: 170, Loss: 1.6218143701553345
Epoch: 180, Loss: 1.63326096534729
Epoch: 190, Loss: 1.646106243133545
Epoch: 200, Loss: 1.5882060527801514
Epoch: 210, Loss: 1.5015379190444946
Epoch: 220, Loss: 1.6467311382293701
Epoch: 230, Loss: 1.4689384698867798
Epoch: 240, Loss: 1.5127537250518799
Epoch: 250, Loss: 1.4545437097549438
Epoch: 260, Loss: 1.3896716833114624
Epoch: 270, Loss: 1.49

Шаблон функции `generate_sample` также доступен ниже. Вы можете как дозаполнить его, так и написать свою собственную функцию с нуля. Не забывайте, что все примеры в обучающей выборке начинались с токена `<sos>`.

In [104]:
def generate_sample(char_rnn, seed_phrase=None, max_length=500, temperature=1.0, device='cuda'):
    char_rnn.eval()

    if seed_phrase is not None:
        x_sequence = [token_to_idx[token] for token in seed_phrase]
    else:
        x_sequence = []

    x_sequence = [token_to_idx['<sos>']] + x_sequence
    x_sequence = torch.tensor([x_sequence], dtype=torch.int64).to(device)

    hidden = None

    while len(x_sequence[0]) - 1 < max_length:
        output, hidden = char_rnn(x_sequence[:, -1].view(1, 1), hidden)

        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]

        x_sequence = torch.cat([x_sequence, top_i.view(1, 1)], dim=1)

    generated_sequence = ''.join([tokens[ix] for ix in x_sequence.cpu().data.numpy()[0]])

    return generated_sequence.replace('<sos>', '', 1)[:max_length]



Пример текста сгенерированного обученной моделью доступен ниже. Не страшно, что в тексте много несуществующих слов. Используемая модель очень проста: это простая классическая RNN.

In [105]:
print(generate_sample(model, ' мой дядя самых честных правил', max_length=500, temperature=0.8))

 мой дядя самых честных правили;
глазамет там беседогольные ларин,
что ее ручей
онегин с изветы, три буян,
как зеленья
взять об одном
и день далеком
ей черернул ее сердце обыта: меж долго в каши двух стол ж из-циила тишин,
унесть одно и то же квакать,
жалеть о прежнем, о бына
на нос и занесены,
любил бы всем стара ж его боят, небеснят награль.
«poor yorick! – молвил он любим… предмете погруженных,
ш остеклет мас. nii

под финского поток
заветный вновь примирать,
брать и чет вечер длинный ножки! 


### Сдача задания
Сгенерируйте десять последовательностей длиной 500, используя строку ' мой дядя самых честных правил'. Температуру для генерации выберите самостоятельно на основании визуального качества генериуремого текста. Не забудьте удалить все технические токены в случае их наличия.

Сгенерированную последовательность сохрание в переменную `generated_phrase` и сдайте сгенерированный ниже файл в контест.

In [106]:
seed_phrase = ' мой дядя самых честных правил'

In [107]:
generated_phrases = [
    generate_sample(
        model,
        seed_phrase,
        max_length=500,
        temperature=0.8
    )
    for _ in range(10)
]

In [109]:
# do not change the code in the block below
# __________start of block__________

if 'generated_phrases' not in locals():
    raise ValueError("Please, save generated phrases to `generated_phrases` variable")

for phrase in generated_phrases:

    if not isinstance(phrase, str):
        raise ValueError("The generated phrase should be a string")

    if len(phrase) != 500:
        raise ValueError("The `generated_phrase` length should be equal to 500")

    assert all([x in set(tokens) for x in set(list(phrase))]), 'Unknown tokens detected, check your submission!'


submission_dict = {
    'token_to_idx': token_to_idx,
    'generated_phrases': generated_phrases
}

np.save('submission_dict_hw07.npy', submission_dict, allow_pickle=True)
print('File saved to `submission_dict_hw07.npy`')
# __________end of block__________

File saved to `submission_dict_hw07.npy`


На этом задание завершено. Поздравляем!